# Лабораторна робота №3
## Варіант №3
Мета роботи:
* Дослідити методи розрахунку ваг альтернатив за критерієм на основі нечітких експертних оцінок парних порівнянь на основі методу нижньої і верхньої апроксимацій LUAM

In [1]:
%pylab inline
import pandas as pd
import itertools
import scipy
import scipy.optimize
from IPython.display import Latex
from IPython.core.display import display, HTML

Populating the interactive namespace from numpy and matplotlib


## Нижня модель LUAM
<img src="./lower_model.png"/>
## Верхня модель LUAM
<img src="./upper_model.png"/>

In [13]:
A = np.array([
    [[1, 1], [3, 4], [6, 6], [6, 7]],
    [[1/4, 1/3], [1, 1], [3, 4], [3, 4]],
    [[1/6, 1/6], [1/4, 1/3], [1, 1], [3, 4]],
    [[1/7,1/6], [1/4, 1/3], [1/4, 1/3], [1, 1]]
])
A = np.array([
    [[1, 1], [1, 3], [3, 5], [5, 7], [5, 9]],
    [[1/3, 1], [1, 1], [1, 4], [1, 5], [1, 4]],
    [[1/5, 1/3], [1/4, 1], [1, 1], [1/5, 5], [2, 4]],
    [[1/7, 1/5], [1/5, 1], [1/5, 5], [1, 1], [1, 2]],
    [[1/9, 1/5], [1/4, 1], [1/4, 1/2], [1/2, 1], [1, 1]]
])
AL = A[:, :, 0]
AU = A[:, :, 1]

def fill_lower(r, a, k, i, j):
    r[k, j + a.shape[1]] = a[i, j]
    r[k, i] = -1

def fill_upper(r, a, k, i, j):
    r[k,  i + a.shape[0]] = 1
    r[k,  j] = -a[i, j]
        
def create_matrix(a, f):
    " w_1^L, ... w_n^L, w_1^U, ..., w_n^[l]"
    n, _ =  a.shape
    r = np.zeros((n * n, 2 * n))
    for k, p in enumerate(itertools.product(range(n), range(n))):
        if (p[0] != p[1]):
            f(r, a, k, p[0], p[1])
    return r

def third_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i] = -1
        for j in range(n):
            if i != j:
                r[i, j + n] = -1
    return r

def fourth_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i + n] = 1
        for j in range(n):
            if i != j:
                r[i, j] = 1
    return r

def fiveth_lower(n):
    r = np.zeros((n, 2 * n))
    for i in range(n):
        r[i, i] = 1
        r[i, i + n] = -1
    return r
        
def solve(l, u, sz=1):
    m, n = l.shape
    s = 2 * n * n
    assert(m == n)
    assert(l.shape == u.shape)
    a1 = sz * create_matrix(l, fill_lower)
    a2 = sz * create_matrix(u, fill_upper)
    a3 = third_lower(n)
    a4 = fourth_lower(n)
    a5 = fiveth_lower(n)
    a_components = [a1, a2, a3, a4, a5]
    a = np.concatenate(a_components)
    b = np.zeros(s + 3 * n)
    b[s:s + n] = -1
    b[s + n: s + 2 * n] = 1
    c = np.zeros(n + n)
    c[0:n] = sz
    c[n:] = -sz
    data = [c, a, b]
    result = scipy.optimize.linprog(
        *data,
        options=dict(disp=True),
        bounds=[(0.0001, np.inf) for i in range(n + n)]
    )
    return data, result

def get_ranges(solution):
    x = solution.x
    n = len(solution.x) // 2
    return list(zip(x[0:n], x[n:]))

In [8]:
def prepare_report(_, solution, n):
    ranges = get_ranges(solution)
    sorted_ranges = sorted(enumerate(ranges), key=lambda x: -(sum(x[1])))
    pattern = "$[w%d^{L}_{%d}, w%d^{U}_{%d}] = [%f, %f]$"
    sorted_ranges_latex = [pattern % (n, i + 1, n, i + 1, rng[0], rng[1]) for i, rng in sorted_ranges]
    J = "$J^{*} = %f$" % np.abs(solution.fun)
    weights = '<br/>'.join(sorted_ranges_latex)
    ordering = "%s" % ("$\succeq$".join([str(i + 1) for i, _ in sorted_ranges]))
    return """
        %s<br/>
        %s<br/>
        %s<br/>
    """ % (J, weights, ordering)

def fuzzy_advantage(a_low, a_mid, a_up):
    a = np.stack((a_low, a_mid, a_up))
    rsi = a.sum(axis=2)
    si = rsi / rsi.sum(axis=1)[::-1, None]
    mm = np.meshgrid(si[1], si[1]) # mm[1] ~ m_i, mm[0] ~ m_j 
    lu = np.meshgrid(si[0], si[2]) # lu[1] ~ u_i, lu[0] ~ l_j
    vss = np.zeros_like(a_low)
    vss[mm[1] >= mm[0]] = 1
    mask_mixed = (mm[1] < mm[0]) * (lu[1] >= lu[0])
    vals_mixed = (lu[1] - lu[0]) / (lu[1] - mm[1] + mm[0] - lu[0])
    vss[mask_mixed] = vals_mixed[mask_mixed]
    print('Matrix S(l,m,u):\n', si.T)
    print('Matrix VS:\n',vss)
    weights = np.amin(vss, axis=1)
    # calculated weights
    return weights / np.linalg.norm(weights, 1)

In [9]:
solution1 = solve(AL, AU, sz=1)
r1 = prepare_report(*solution1, 1)
solution2 = solve(AL, AU, sz=-1)
r2 = prepare_report(*solution2, 2)

Optimization terminated successfully.
         Current function value: -0.223535   
         Iterations: 45
Optimization terminated successfully.
         Current function value: 0.618182    
         Iterations: 23


# Розв'язок нижньої моделі

In [11]:
HTML(r1)

# Розв'язок верхньої моделі

In [12]:
HTML(r2)

In [22]:
fuzzy_advantage(AL, (AL + AU) / 2, AU)

Matrix S(l,m,u):
 [[ 0.23352361  0.43539484  0.90457456]
 [ 0.06746238  0.21044084  0.54274474]
 [ 0.05682408  0.16309165  0.4100738 ]
 [ 0.03958781  0.12781948  0.33288344]
 [ 0.03286629  0.06325319  0.13387704]]
Matrix VS:
 [[ 1.          1.          1.          1.          1.        ]
 [ 0.57887594  1.          1.          1.          1.        ]
 [ 0.393336    0.87857956  1.          1.          1.        ]
 [ 0.24416623  0.76261125  0.88670545  1.          1.        ]
 [ 0.          0.3109267   0.43559467  0.59355336  1.        ]]


array([ 0.45118654,  0.26118103,  0.17746791,  0.11016452,  0.        ])